In [4]:
from Utils.Utils import *
from Utils.Blacksmith import * 

from Utils.HyMNet import HyMNet

In [5]:
# Seed
set_seed(0)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
PATH = "/home/baharoon/HTN/data/"
CSV_PATH = {"HTNPath": PATH + r"HTN", "NonHTNPath": PATH + "NonHTN"}

MODELS_PATH = "/home/baharoon/HTN/Models"

os.makedirs(MODELS_PATH, exist_ok=True)

In [8]:
BATCH_SIZE = 16

train_transform = T.Compose([
    T.Resize((256, 256)),
    T.CenterCrop(224),
    T.ToTensor(),
    T.RandomHorizontalFlip(0.5),
    T.RandomRotation(degrees=(0, 360)),
    T.GaussianBlur(3),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = T.Compose([
    T.Resize((256, 256)),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = HypertensionDataset(CSV_PATH, split="train", train_transform=train_transform, tabular_only=True)
val_dataset = HypertensionDataset(CSV_PATH, split="val", test_transform=test_transform, tabular_only=True)

train_x, train_y = train_dataset.df[["Age", "Gender"]], train_dataset.df["HTN"]
val_x, val_y = val_dataset.df[["Age", "Gender"]], val_dataset.df["HTN"]

train_x, train_y = pd.concat([train_x, val_x]), pd.concat([train_y, val_y])

/home/baharoon/HTN/HyMNet/Utils/Utils.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HTN_DM'] = df['HTN'].astype(str) + df['DM'].astype(str)
/home/baharoon/HTN/HyMNet/Utils/Utils.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HTN_DM'] = df['HTN'].astype(str) + df['DM'].astype(str)


# XGBoost

In [18]:
boost = xgb.XGBClassifier(device="cuda", objective="binary:logistic")

In [19]:
param_grid = {'max_depth': [3,6,10],
           'learning_rate': [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.3],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

clf = GridSearchCV(estimator=boost, 
                   param_grid=param_grid,
                   scoring='roc_auc', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed:  3.6min finished


0.6876565784239242


In [21]:
with open(MODELS_PATH+'/DemographicXGBParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

# SVM

In [42]:
from sklearn.svm import SVC
svm = SVC()

In [44]:
param_grid = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']},
    {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'degree': [1,2,3], 'kernel': ['poly']}
]

clf = GridSearchCV(estimator=svm, 
                   param_grid=param_grid,
                   scoring='roc_auc', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

Fitting 5 folds for each of 68 candidates, totalling 340 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.7003619518563511


[Parallel(n_jobs=1)]: Done 340 out of 340 | elapsed:  3.3min finished


In [46]:
with open(MODELS_PATH+'/DemographicSVMParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

# FCNN

In [6]:
results = {}

In [7]:
lrs = [0.005]
criterion = nn.BCEWithLogitsLoss()
epochs = 250
epoch_length = math.ceil(len(train_dataset) / BATCH_SIZE)

In [8]:
for lr in lrs:
    tabular_model = nn.Sequential(
        nn.Linear(in_features=2, out_features=8),
        nn.LeakyReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=8, out_features=32),
        nn.LeakyReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=32, out_features=16),
        nn.LeakyReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=16, out_features=8),
        nn.LeakyReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=8, out_features=1),
    ).to(device)
        
    model = HyMNet(tabular_model=tabular_model)    
    
    optimizer = torch.optim.Adam(tabular_model.to(device).parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epoch_length * epochs, eta_min=0)
    metrics, best_model = train_val(epochs=epochs, model=model, criterion=criterion, optimizer=optimizer, train_loader=train_loader,
                        val_loader=val_loader, scheduler=scheduler, device=device, save_model=True)
    results[lr] = max([metrics[1][i]["AUROC"] for i in metrics[1]])

Beginning Training: 

Epoch 1/250


Batch in Progress: 100%|█████████████████████████████████████████| 251/251 [00:01<00:00, 126.17it/s]


Train Set:
    Average Loss:  0.666033,    Accuracy: 60.56%,    Correct Counter: 2428/4009,    F1 Score: 0.73,    Precision: 0.61,    Recall:  0.92,    AUROC:  0.54



Batch in Progress: 100%|███████████████████████████████████████████| 64/64 [00:00<00:00, 219.27it/s]


Validation Set:
    Average Loss:  0.621554,    Accuracy: 67.82%,    Correct Counter: 685/1010,    F1 Score: 0.75,    Precision: 0.66,    Recall:  0.88,    AUROC:  0.65

Epoch 2/250


...

Validation Set:
    Average Loss:  0.597378,    Accuracy: 67.82%,    Correct Counter: 685/1010,    F1 Score: 0.76,    Precision: 0.65,    Recall:  0.93,    AUROC:  0.65

Epoch 250/250


Batch in Progress: 100%|█████████████████████████████████████████| 251/251 [00:01<00:00, 182.44it/s]


Train Set:
    Average Loss:  0.611129,    Accuracy: 67.47%,    Correct Counter: 2705/4009,    F1 Score: 0.77,    Precision: 0.66,    Recall:  0.92,    AUROC:  0.63



Batch in Progress: 100%|███████████████████████████████████████████| 64/64 [00:00<00:00, 206.36it/s]

Validation Set:
    Average Loss:  0.597377,    Accuracy: 67.82%,    Correct Counter: 685/1010,    F1 Score: 0.76,    Precision: 0.65,    Recall:  0.93,    AUROC:  0.65



In [16]:
torch.save(best_model, MODELS_PATH + r'/DemographicFCNN.pth')

In [13]:
with open("/home/baharoon/HTN/HyMNet/Results/demographicfcnn.json", 'w') as f:
    # Use json.dump to write the dictionary to the file
    json.dump(results, f)